# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# your code here

orders = pd.read_csv("/Users/medea/Documents/Ironhack/Labs_Week_2/lab-dataframe-calculations/your-code/Orders.csv")

orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# your code here

# sub problem 1

amount_spent_customers = pd.DataFrame(orders.groupby(["CustomerID"]).agg({"amount_spent": "sum"}))

amount_spent_customers

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [5]:
# sub problem 2 & 3

labels = ["Basic", "Standard", "Preferred", "VIP"]

amount_spent_customers_quantile = pd.qcut(amount_spent_customers["amount_spent"], 4, labels = labels)

amount_spent_customers_quantile


CustomerID
12346         VIP
12347         VIP
12348         VIP
12349         VIP
12350    Standard
           ...   
18280       Basic
18281       Basic
18282       Basic
18283         VIP
18287         VIP
Name: amount_spent, Length: 4339, dtype: category
Categories (4, object): ['Basic' < 'Standard' < 'Preferred' < 'VIP']

In [6]:
amount_spent_customers_quantile.value_counts()

VIP          1085
Standard     1085
Basic        1085
Preferred    1084
Name: amount_spent, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [7]:
# your code here


orders_labeled = pd.merge(left= orders, right= amount_spent_customers_quantile, how= "inner", left_on= "CustomerID", right_on= "CustomerID")

orders_labeled.rename(columns = {"amount_spent_y": "status", "amount_spent_x": "amount_spent"}, inplace= True)

orders_labeled.head(2)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,status
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,VIP
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP


In [8]:


condition = orders_labeled["status"] == "VIP"

orders_labeled[condition][["Country", "status"]].value_counts()


Country          status
United Kingdom   VIP       221635
Germany          VIP         7349
EIRE             VIP         7238
France           VIP         6301
Netherlands      VIP         2080
Spain            VIP         1569
Belgium          VIP         1557
Switzerland      VIP         1370
Portugal         VIP         1093
Australia        VIP         1028
Norway           VIP         1028
Channel Islands  VIP          589
Italy            VIP          507
Finland          VIP          504
Cyprus           VIP          451
Sweden           VIP          273
Japan            VIP          272
Singapore        VIP          222
Denmark          VIP          217
Israel           VIP          214
Iceland          VIP          182
Austria          VIP          158
Poland           VIP          149
Canada           VIP          135
Malta            VIP           67
Lebanon          VIP           45
Greece           VIP           32
dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [10]:
# your code here


condition = (orders_labeled["status"] == "VIP" ) | (orders_labeled["status"] == "Preferred")

orders_labeled[condition][["Country", "status"]].value_counts()

Country               status   
United Kingdom        VIP          221635
                      Preferred     77410
Germany               VIP            7349
EIRE                  VIP            7238
France                VIP            6301
Netherlands           VIP            2080
Spain                 VIP            1569
Belgium               VIP            1557
Switzerland           VIP            1370
France                Preferred      1112
Portugal              VIP            1093
Norway                VIP            1028
Australia             VIP            1028
Germany               Preferred       926
Spain                 Preferred       659
Channel Islands       VIP             589
Italy                 VIP             507
Finland               VIP             504
Cyprus                VIP             451
Switzerland           Preferred       381
Belgium               Preferred       364
Sweden                VIP             273
Japan                 VIP             272
Si